In [1]:
import numpy as np

In [2]:
deck = np.repeat(np.array(['2','3','4','5','6','7','8','9','10','J','Q','K','A']), 4)
values = np.array([2,3,4,5,6,7,8,9,10,10,10,10,11])
dict_map = dict(zip(deck, values))
n_decks = 8
stream = np.repeat(deck, n_decks)

In [3]:
rng = np.random.default_rng() # https://numpy.org/doc/stable/reference/random/index.html
rng.shuffle(stream)

In [8]:
%timeit rng.shuffle(stream)

7.96 µs ± 62.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
